# Part 3. dtype(자료형): 전처리의 핵심 엔진
이 장은 전처리에서 가장 중요한 판단 기준인 dtype를 다룬다. <br>
dtype는 단순한 자료형 정보가 아닌 "이 컬럼을 계산·비교·분석할 수 있는가"를 결정하는 규칙이다.

## 1) dtype가 중요한 이유
dtype는 Pandas가 각 컬럼을 어떻게 해석하고 처리할지 결정하는 기준이다.

같은 값이라 하더라도 dtype가 다르면:
- 계산 결과가 달라지고
- 정렬 결과가 달라지고
- 필터링 결과가 달라진다

### 민약 dtype가 틀리다면?
dtype가 틀린 상태에서는 겉보기에는 멀쩡해 보여도 분석이 정상적으로 작동하지 않는다.

1. 분명 보이는건 숫자인데 문자열이여서 계산이 안 된다
2. 날짜인데 문자열이여서 날짜 연산이 안 된다
3. True/False인데 필터가 이상하다

이런 문제들은 에러를 내지 않고 조용히 잘못된 결과를 만든다.

### 만약 dtype가 맞다면?
전처리에서 중요한 건 "**원본 dtype**"이 아닌 "**분석 가능한 dtype**" 이다.
- 가격, 수량 → 숫자형 (int / float)
- 날짜 → datetime
- 상태값 → bool 또는 범주형

dtype가 맞아야 그 다음 단계인 집계, 정렬, 필터링, 시각화가 의도한 대로 이어진다.<br>
때문에 항상 질문하는 습관을 가지자

## 2) dtype 변환의 두 가지 방식

In [1]:
import pandas as pd

df = pd.DataFrame({
    "date": ["2026-01-01", "2026/01/02", "2026-01-03"],
    "menu": ["Americano", "Latte", "Mocha"],
    "price": ["4500원", "5,000", None],
    "qty": ["2", 1, None],
    "paid": ["TRUE", "FALSE", True]
})

print("원본 DataFrame")
print(df)

print("\nDataFrame 정보")
print(df.info())

원본 DataFrame
         date       menu  price   qty   paid
0  2026-01-01  Americano  4500원     2   TRUE
1  2026/01/02      Latte  5,000     1  FALSE
2  2026-01-03      Mocha   None  None   True

DataFrame 정보
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    3 non-null      object
 1   menu    3 non-null      object
 2   price   2 non-null      object
 3   qty     2 non-null      object
 4   paid    3 non-null      object
dtypes: object(5)
memory usage: 248.0+ bytes
None


### 1. astype() — 강제 변환
astype는 값의 의미를 해석하지 않고, 형태만 강제로 바꾼다.

- 이미 값이 깔끔할 때 사용한다:
    - 문자열이지만 실제로는 전부 숫자일 때
    - True / False 값이 일관되게 들어 있을 때
- 실패하면 에러가 발생, 값이 깨질 수 있다
> 즉, "이 값은 이미 이 타입이 맞다"는 확신이 있을 때만 사용한다.

기본형태
``` python
컬럼.astype("string")
```
int, float, bool 도 가능

In [2]:
print("qty 컬럼을 astype(int)로 강제 변환 시도")
#df["qty"].astype(int)
print("실패하는 이유: 결측값이 존재하기 때문")

print("\nprice 컬럼을 astype(int)로 강제 변환 시도")
#df["price"].astype(int)
print("실패하는 이유: ""4500원"", ""5,000"" → 숫자로 해석 불가")

print("\nprice 컬럼을 astype(string)로 강제 변환 시도")
df["paid"] = (
    df["paid"]
    .astype("string")
    .str.upper() # Series에 문자열 전용 메소드를 쓰겠다는 접근자
    .map({"TRUE": True, "FALSE": False}) # 의미 → 값 매핑
)
df["paid"]

qty 컬럼을 astype(int)로 강제 변환 시도
실패하는 이유: 결측값이 존재하기 때문

price 컬럼을 astype(int)로 강제 변환 시도
실패하는 이유: 4500원, 5,000 → 숫자로 해석 불가

price 컬럼을 astype(string)로 강제 변환 시도


0     True
1    False
2     True
Name: paid, dtype: bool

### 2. 파싱 변환 — 해석해서 바꾸는 방식
파싱 변환은 문자열 안의 내용을 해석해서 적절한 dtype로 바꾼다. <br>
이 방식은:
- 단위가 섞여 있거나
- 공백, 쉼표, 문자가 포함되어 있거나
- 일부 값이 깨져 있을 때

가장 안전하다.

#### to_datetime 문장 예시
``` python
# to_datetime
pd.to_datetime(컬럼, errors="coerce")
```
날짜 문자열을 datetime으로 해석 <br>
날짜처럼 보이는 값으로 월별, 요일별, 기간 분석을 하고 싶을 때 사용된다.

In [3]:
print("date 컬럼을 datetime으로 파싱 변환")
df["date"] = pd.to_datetime(df["date"], errors="coerce")
print(df["date"])

date 컬럼을 datetime으로 파싱 변환
0   2026-01-01
1          NaT
2   2026-01-03
Name: date, dtype: datetime64[ns]


#### to_numeric 문장 예시
```python
# to_numeric
pd.to_numeric(컬럼, errors="coerce")
```
숫자로 해석 가능한 값만 숫자로 변환 (전처리에서 가장 많이 씀) <br>
이 컬럼은 숫자처럼 보이지만 실제로는 문자열일 때 사용된다.

In [4]:
print("qty 컬럼을 to_numeric으로 파싱 변환")
df["qty"] = pd.to_numeric(df["qty"], errors="coerce")
print(df["qty"])

qty 컬럼을 to_numeric으로 파싱 변환
0    2.0
1    1.0
2    NaN
Name: qty, dtype: float64


In [5]:
# 해당 컬럼이 단순 문자열만 있는것이 아닌 경우
print("price 컬럼 문자열 정리")
df["price"] = (
    df["price"]
    .astype("string")
    .str.replace(",", "", regex=False)
    .str.replace("원", "", regex=False)
)
df["price"] = pd.to_numeric(df["price"], errors="coerce")
print(df["price"])

price 컬럼 문자열 정리
0    4500
1    5000
2    <NA>
Name: price, dtype: Int64


## 3) 변환 실패를 처리하는 errors 옵션
dtype 변환에서는 항상 변환에 실패하는 값이 생길 수 있다. <br>
이를 어떻게 처리할지 정하는 것이 errors 옵션이다.

이 옵션의 핵심 의미는 "문제를 숨기는 것"이 아닌 문제를 눈에 보이게 만드는 점이다.
- 어디서 변환이 실패했는지 확인할 수 있다
- 이후 결측치 처리 전략으로 이어질 수 있다

### (1) errors="raise"
실패하면 즉시 중단
```python
pd.to_numeric(컬럼, errors="raise")
pd.to_datetime(컬럼, errors="raise")
```
변환에 실패하는 값이 하나라도 있으면 에러를 발생시키고 작업을 중단한다
- 데이터 검증 단계에는 유용
- 실무 전처리에는 거의 쓰이지 않음

### (2) errors="coerce"
실패하면 결측치로 바꾸고 계속 진행
```python
pd.to_numeric(컬럼, errors="coerce")
pd.to_datetime(컬럼, errors="coerce")
```
- 변환이 안 되는 값은 문제가 있는 데이터로 표시하고 일단 분석 흐름을 유지한다
    - 숫자 변환 실패 → NaN
    - 날짜 변환 실패 → NaT

문제가 되는 값은 바로 볼수 있기 때문에 다음 단계에서 의도적으로 처리한다.

In [6]:
print("qty의 결측값 확인")
print(df["qty"])

print("\ndate의 결측값 확인")
print(df["date"])

print("\nprice의 결측값 확인")
print(df["price"])

qty의 결측값 확인
0    2.0
1    1.0
2    NaN
Name: qty, dtype: float64

date의 결측값 확인
0   2026-01-01
1          NaT
2   2026-01-03
Name: date, dtype: datetime64[ns]

price의 결측값 확인
0    4500
1    5000
2    <NA>
Name: price, dtype: Int64


## 4) dtype 변환 후 반드시 다시 확인해야 하는 이유

In [7]:
print("바뀐 데이터프레임의 정보")
df.info()

바뀐 데이터프레임의 정보
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    2 non-null      datetime64[ns]
 1   menu    3 non-null      object        
 2   price   2 non-null      Int64         
 3   qty     2 non-null      float64       
 4   paid    3 non-null      bool          
dtypes: Int64(1), bool(1), datetime64[ns](1), float64(1), object(1)
memory usage: 230.0+ bytes


dtype 변환으로 qty, date, price, paid 4개의 데이터를 바꾸었으니 해당 컬럼의 타입을 확인하자.
- date → datetime64
- price, qty → float64 (결측치가 있으면 int가 아니라 float로 보이는 것이 정상)
- paid → bool

의도한대로 잘 바뀌었다고 볼수 있으나, 아직 처리해야할 문제<br>
**결측치**가 남아있다